<a href="https://colab.research.google.com/github/subh007/ml-ground/blob/main/time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install tensorflow

In [45]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("robervalt/sunspots")

print("Path to dataset files:", path)

100%|██████████| 22.4k/22.4k [00:00<00:00, 22.5MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/robervalt/sunspots/versions/3


In [4]:
import pandas as pd

In [5]:
df = pd.read_csv(path + "/Sunspots.csv")

In [6]:
df.head()

,Unnamed: 0,Date,Monthly Mean Total Sunspot Number
0,0,1749-01-31,96.7
1,1,1749-02-28,104.3
2,2,1749-03-31,116.7
3,3,1749-04-30,92.8
4,4,1749-05-31,141.7


In [7]:
df.shape

(3265, 3)

In [8]:
df.size

9795

In [73]:
dataset = tf.keras.utils.timeseries_dataset_from_array(df['Monthly Mean Total Sunspot Number'][:-3], df['Monthly Mean Total Sunspot Number'][3:],sequence_length=3, batch_size=2)

In [74]:
dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>

In [96]:
x_data = []
y_data = []
for X, y in dataset:
  x_data.append(X.numpy())
  y_data.append(y.numpy())
  # print("X (Input):", X.numpy(), "→ y (Future Target):", y.numpy())

In [97]:
# Concatenate all batches along the first dimension
X_all = np.concatenate(x_data, axis=0)
y_all = np.concatenate(y_data, axis=0)

In [98]:
print("Total X shape:", X_all.shape)  # e.g., (number_of_samples, sequence_length)
print("Total y shape:", y_all.shape)  # e.g., (number_of_samples,)

Total X shape: (3260, 3)
Total y shape: (3260,)


In [99]:
X_all[0]

array([ 96.7, 104.3, 116.7])

In [87]:
y_data[0]

array([ 92.8, 141.7])

In [101]:
train_x, test_x, train_y, test_y = train_test_split(X_all, y_all, test_size=0.2)

In [102]:
print("Total X shape:", train_x.shape)  # e.g., (number_of_samples, sequence_length)
print("Total y shape:", train_y.shape)  # e.g., (number_of_samples,)
print("Total X shape:", test_x.shape)  # e.g., (number_of_samples, sequence_length)
print("Total y shape:", test_y.shape)  # e.g., (number_of_samples,)

Total X shape: (2608, 3)
Total y shape: (2608,)
Total X shape: (652, 3)
Total y shape: (652,)


In [111]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(128, activation='relu', input_shape=(3,)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(1))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [112]:
model.compile(optimizer=tf.keras.optimizers.Adadelta(), loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy'])

In [113]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ batch_normalization                  │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [114]:
model.fit(train_x, train_y, epochs=10, batch_size=32, validation_data=(test_x, test_y), verbose=1)

Epoch 1/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.0195 - loss: 12049.0752 - val_accuracy: 0.0169 - val_loss: 11569.5332
Epoch 2/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0214 - loss: 10816.5117 - val_accuracy: 0.0169 - val_loss: 11561.5732
Epoch 3/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0206 - loss: 11207.0342 - val_accuracy: 0.0169 - val_loss: 11558.7197
Epoch 4/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0248 - loss: 11420.0859 - val_accuracy: 0.0169 - val_loss: 11556.6621
Epoch 5/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0229 - loss: 11196.6113 - val_accuracy: 0.0169 - val_loss: 11554.7334
Epoch 6/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0226 - loss: 11427.7803 - val_accuracy: 0.0169 - val_loss: 11552.7480
Epoch 7/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0217 - loss: 11335.1865 - val_accuracy: 0.0169 - val_loss: 11550.6807
Epoch 8/10
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy:

In [109]:
# Reshape the input array to (1, 3) by adding a batch dimension
input_array = np.array([12, 23, 34])
reshaped_input = input_array.reshape(1, 3)  # Reshape from (3,) to (1, 3)

# Now, predict using the reshaped input
predictions = model.predict(reshaped_input)
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
[[-4.9657907]]


In [110]:
# Now, predict using the reshaped input
predictions = model.predict(reshaped_input)
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
[[-4.9657907]]
